# debug web
#https://pythononpow.medium.com/debugging-a-tornado-pythononwheels-web-application-with-vscode-d9f5c5e5603f

In [2]:
# {
#     // 使用 IntelliSense 了解相关属性。 
#     // 悬停以查看现有属性的描述。
#     // 欲了解更多信息，请访问: https://go.microsoft.com/fwlink/?linkid=830387
#     "version": "0.2.0",
#     "configurations": [
        
#         {  
#             "name": "Python: PoW",  
#             "type": "python",  
#             "request": "launch",  
#             "program": "${workspaceFolder}/instock/web/web_service.py",  
#             "args": [],  
#             "console": "integratedTerminal"  
#         }
            
#     ]
# }

In [3]:
import logging
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
from sqlalchemy import inspect
import datetime

import sys

# stdout=sys.stdout
sys.path.append('../../')
from JohnsonUtil import commonTips as cct
from JSONData import tdx_data_Day as tdd

# 初始化db

In [4]:
db_host = "192.168.50.60"  # 数据库服务主机
db_user = "root"  # 数据库访问用户
db_password = "mariadb"  # 数据库访问密码
db_database = "instockdb"  # 数据库名称
db_port = 3306  # 数据库服务端口
db_charset = "utf8mb4"  # 数据库字符集

# 使用环境变量获得数据库,docker -e 传递
_db_host = os.environ.get('db_host')
if _db_host is not None:
    db_host = _db_host
_db_user = os.environ.get('db_user')
if _db_user is not None:
    db_user = _db_user
_db_password = os.environ.get('db_password')
if _db_password is not None:
    db_password = _db_password
_db_database = os.environ.get('db_database')
if _db_database is not None:
    db_database = _db_database
_db_port = os.environ.get('db_port')
if _db_port is not None:
    db_port = int(_db_port)

MYSQL_CONN_URL = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    db_user, db_password, db_host, db_port, db_database, db_charset)

MYSQL_CONN_DBAPI = {'host': db_host, 'user': db_user, 'password': db_password, 'database': db_database,
                    'charset': db_charset, 'port': db_port, 'autocommit': True}



## def func

In [5]:
def write_code_to_blk(codew, blk='060'):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
    if write_blk == 'y' or write_blk == 'Y':
        hdf5_wri = cct.cct_raw_input(
            "Rewrite code [Y] or append [N](defalut:N):")
        if hdf5_wri == 'y' or hdf5_wri == 'Y':
            append_status = False
        else:
            append_status = True
        if len(codew) > 3:
            cct.write_to_blocknew(block_path, codew, append_status,
                                  doubleFile=False, keep_last=0)
            print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write No:%s block_path:%s" % (len(codew), block_path))
            # ("write error:%s block_path:%s" % (len(codew), block_path))
    return True

#allday
def groupby_count_code(df):
    df['couts'] = df.groupby('code')['code'].transform('count')
    if 'date' in df.columns:
        df = df.sort_values(by=['couts','date'],ascending=[0,1])
    else:
        df = df.sort_values(by=['couts'],ascending=[0])
    return df

In [6]:
def get_today(sep='-'):
    TODAY = datetime.date.today()
    fstr = "%Y" + sep + "%m" + sep + "%d"
    today = TODAY.strftime(fstr)
    return today

#前一日
def day_last_days(daynow,last=-1):
    return str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() + datetime.timedelta(last))

#上一个工作日
def last_tddate(days=1):
    # today = datetime.datetime.today().date() + datetime.timedelta(-days)
    if days is None:
        return days
    today = datetime.datetime.today().date()
    # log.debug("today:%s " % (today))
    # return str(today)

    def get_work_day(today):
        day_n = int(today.strftime("%w"))
        if day_n == 0:
            lastd = today + datetime.timedelta(-2)
            # log.debug("0:%s" % lastd)
        elif day_n == 1:
            lastd = today + datetime.timedelta(-3)
            # log.debug("1:%s" % lastd)
        else:
            lastd = today + datetime.timedelta(-1)
            # log.debug("2-6:%s" % lastd)
        return lastd
        # if days==0:
        # return str(lasd)
    lastday = today
    for x in range(int(days)):
        # print x
        lastday = get_work_day(today)
        today = lastday
    return str(lastday)

# 通过数据库链接 engine
def engine():
    return create_engine(MYSQL_CONN_URL)


def engine_to_db(to_db):
    _engine = create_engine(MYSQL_CONN_URL.replace(f'/{db_database}?', f'/{to_db}?'))
    return _engine


# DB Api -数据库连接对象connection，有游标
def conn_with_cursor():
    return conn_not_cursor().cursor()

# DB Api -数据库连接对象connection，无游标
def conn_not_cursor():
    try:
        _db = pymysql.connect(**MYSQL_CONN_DBAPI)
    except Exception as e:
        logging.error(f"database.conn_not_cursor处理异常：{MYSQL_CONN_DBAPI}{e}")
    return _db

# 查询数据
def executeSqlFetch(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.executeSqlFetch处理异常：{sql}{e}")

        result = db.fetchall()
        db.close()
        return result
    
# 计算数量
def executeSqlCount(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.select_count计算数量处理异常：{e}")

        result = db.fetchall()
        db.close()
        # 只有一个数组中的第一个数据
        if len(result) == 1:
            return int(result[0][0])
        else:
            return 0

In [7]:
try:
    with pymysql.connect(**MYSQL_CONN_DBAPI) as mydb:
        mydb.cursor().execute(" select 1 ")
except Exception as e:
        logging.error("执行信息：数据库不存在，将创建。")

## 放量上涨table 
cn_stock_strategy_entercn_stock_strategy_enter

In [8]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')


def panda_df(conn,sql):
    df = pd.read_sql(sql, conn)
    print(df.shape)
    return df

from pandas import DataFrame as df
def exe_sql_select(conn,sql):
	cur = conn.cursor() 
	data = None
	try: # 使用异常处理，以防程序无法正常运行
		cur.execute(sql) 
		data = df(cur.fetchall(), columns = [col[0] for col in cur.description]) 
	except Exception as e:
		conn.rollback() # 发生错误时回滚
		print('事务处理失败', e)
	else:
		# conn.commit() # 事务提交
		print('事务处理成功', cur.rowcount)
	# cur.close()
	return data



### 读取SQL数据

In [9]:
conn = pymysql.connect(**MYSQL_CONN_DBAPI)

_selcolall = f''' `date`,`code`,`name`,`rate_1` '''
_selcol = f''' `code`,`name`,`rate_1` '''

_table_name = 'cn_stock_strategy_enter'

lastday = last_tddate(1)

today_now = last_tddate(0)



sql = f'''SELECT '{_selcol}' FROM `{_table_name}` WHERE `date` = '{today_now}' and 
								`kdjk` >= 80 and `kdjd` >= 70 and `kdjj` >= 100 and `rsi_6` >= 80 and 
								`cci` >= 100 and `cr` >= 300 and `wr_6` >= -20 and `vr` >= 160'''

sql_now_price = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' AND rate_1 > 0 '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' '''


sql_now_last7day = f'''SELECT {_selcolall} FROM `{_table_name}` WHERE `date` >= '{last_tddate(7)}' '''

sql_now_alltoday = f'''SELECT {_selcolall} FROM `{_table_name}`  '''

sql_tables_name =f'''select table_name from information_schema.tables where table_schema=`{db_database}` '''
# sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where TABLE_SCHEMA=`{db_database}` and table_name = `{_table_name}`'''

sql_tables_name ='SHOW DATABASES'
# print('当前库下所有表的名称')
# sql_columns = "select COLUMN_NAME from information_schema.COLUMNS where table_name = '%s'"%(_table_name)
sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where table_name = '{_table_name}' '''
print(sql_now_price)
print(sql_now_today)

mycursor = conn.cursor()
# mycursor.execute(sql_now_price)
# print(_selcol)
# count = 0
# for field in mycursor.fetchall():
# 		# print(field[0],end='  ')
# 		# print(field[1:4])
# 		# print(field)
# 		count +=1
# print("all:%s"%(count))


#lastday 
# df = exe_sql_select(conn,sql_now_price)

#today:
df = exe_sql_select(conn,sql_now_today)

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(conn,sql_now_today)

df3all = panda_df(conn,sql_now_alltoday)

dflast7d = panda_df(conn,sql_now_last7day)

# mycursor.execute(sql_columns)
# mycursor.execute(sql_now)

# myresult = mycursor.fetchall()
# for x in myresult:
#   print(x)
	
conn.close()

SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-08-21' AND rate_1 > 0 
SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-08-22' 
事务处理成功 19
(19, 3)
(2223, 4)
(84, 4)


In [10]:
#today
print(df2[:2],df2.shape)
vol_code = df2.code.tolist()[:2]
#allday
df2[:2]
dflast7d[:2]


     code  name  rate_1
0  000034  神州数码    2.78
1  000603  盛达资源    6.39 (19, 3)


,date,code,name,rate_1
0,2023-08-11,000029,深深房Ａ,-2.80
1,2023-08-11,000705,浙江震元,0.97


In [11]:
# df3all = groupby_count_code(df3all)
dflast7d = groupby_count_code(dflast7d)

### show couts

In [12]:
filter_date = '2023-04-28'
filter_date = today_now
# last_tddate(0)
# filter_date = lastday

In [13]:
df7multiIndex = dflast7d.reset_index().set_index(['code','date'])
df7multiIndex[:5]
df7tail = df7multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])

### show Today

In [14]:
df7tail =  df7tail.reset_index().set_index('code')
# df7tail[ df7tail.rate_1 ] 
df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())][:6]

,date,index,name,rate_1,couts
code,,,,,
300541,2023-08-22,72,先进数通,5.34,2
603985,2023-08-22,83,恒润股份,3.84,2
300588,2023-08-22,74,熙菱信息,2.67,2
300603,2023-08-22,75,立昂技术,2.61,2


In [15]:
df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ][:10]

,date,index,name,rate_1,couts
code,,,,,
300226,2023-08-22,69,上海钢联,20.00,1
301169,2023-08-22,78,零点有数,20.00,1
300465,2023-08-22,71,高伟达,11.18,1
301339,2023-08-22,79,通行宝,8.46,1
603629,2023-08-22,82,利通电子,6.78,1
300895,2023-08-22,76,铜牛信息,6.41,1
000603,2023-08-22,66,盛达资源,6.39,1
300561,2023-08-22,73,汇金科技,6.19,1
002025,2023-08-22,68,航天电器,6.02,1


In [16]:
# df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])
# df7tail

### 写入放量上涨061

In [17]:
# df7code = df7tail[df7tail.couts >1].index.get_level_values('code')
# filter_date = '2023-04-28' couts >1

df7code = df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())].index

#nowday
df7code = df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ].index
df7code
len(df7code)

19

In [18]:

# hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
# if len(hdf5_wri) == 0:
#     pass
# elif hdf5_wri == 'y' or hdf5_wri == 'Y':
#     append_status = False
# else:
#     append_status = True
# df2.code.values
#today:
# write_code_to_blk(df7code,'061')

# write_code_to_blk(df7code,'062')

# write_code_to_blk(df7code,'064')


write blk [Y] or [N]: n


True

## 读取cn_stock_strategy_keep_increasing

In [19]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_strategy_keep_increasing'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = '2023-05-08'
start_d = last_tddate(5)
# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` > '{today_now}'- INTERVAL 5 DAY '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df_increas = panda_df(engine(),sql_now_today)

df_increasA = panda_df(engine(),sql_now_allday)

2023-08-22
(3, 103)
(18, 103)


In [20]:
df_increas = df_increas.sort_values(by=['rate_1'],ascending=[1])
df_increas

,date,code,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,...,rate_91,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100
0,2023-08-22,300603,立昂技术,2.61,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,2023-08-22,300895,铜牛信息,6.41,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2023-08-22,301169,零点有数,20.00,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## 写入周063

In [21]:
# write_code_to_blk(df_increas.code.tolist(),'063')

### Filter Day多日

In [22]:
df_increas.shape
df_increasA.shape
df_increasA.columns[:10]
# df_iA = df_increasA.groupby(level=[0]).tail(1)
df_iA = groupby_count_code(df_increas)
df_iA_multiIndex = df_iA.reset_index().set_index(['code','date'])
df_iA_multiIndex.shape
# df_iA_multiIndex[:10]
# df_iA_multiIndex[-10:]
# df_iA.loc[df_iA.code == '000028']
df_iA_multiIndex[df_iA_multiIndex.index.get_level_values('code') == '000028'][:2]

,,index,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,rate_8,...,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100,couts
code,date,,,,,,,,,,,,,,,,,,,,,


In [23]:
df_iAtail = df_iA_multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df_iAtail =  df_iAtail.reset_index().set_index('code')
df_iAtail.loc[:,['date','name','couts']][:5]
# df_iA.loc[:,['date','code','name','couts']][:50]

# df_iAtail[df_iAtail.couts < 3].loc[:,['date','name','couts']]

,date,name,couts
code,,,
300603,2023-08-22,立昂技术,1
300895,2023-08-22,铜牛信息,1
301169,2023-08-22,零点有数,1


### show Couts

In [24]:
df_iA.loc[:,['date','code','name','couts']]

,date,code,name,couts
0,2023-08-22,300603,立昂技术,1
1,2023-08-22,300895,铜牛信息,1
2,2023-08-22,301169,零点有数,1


## cn_stock_indicators MutilIndex

### 读取SQL数据表

In [25]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_indicators'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = last_tddate(5)

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(engine(),sql_now_today)

# df2 = panda_df(engine(),sql_now_allday)



# conn.close()

2023-08-22
(26907, 77)


In [26]:
df2.columns
df2.loc[df2.code =='000061'][:5]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
43,2023-08-15,000061,农 产 品,6.97,0.123117,0.164935,-0.041818,37.0872,41.9064,27.44890,...,-0.001031,0.131914,1411320.0,981885.0,208767.0,7.79331,7.09121,6.38911,25.3538,12.56470
4526,2023-08-16,000061,农 产 品,6.84,0.105218,0.152927,-0.047709,32.7495,38.8541,20.54040,...,-0.064155,0.046274,-1811240.0,-880197.0,-79808.9,7.75113,7.05283,6.35453,0.0000,12.58710
9010,2023-08-17,000061,农 产 品,6.93,0.097212,0.141777,-0.044564,36.2121,37.9734,32.68940,...,-0.085938,-0.022418,1035550.0,396967.0,79528.0,7.72005,7.02455,6.32905,14.7112,13.36870
13494,2023-08-18,000061,农 产 品,6.82,0.081014,0.129563,-0.048549,31.3309,35.7593,22.47430,...,-0.079777,-0.063499,-883883.0,-456933.0,-58101.6,7.69230,6.99930,6.30630,0.0000,4.90042
17979,2023-08-21,000061,农 产 品,6.75,0.061764,0.115886,-0.054122,23.5017,31.6734,7.15822,...,-0.071492,-0.077846,-493493.0,-481306.0,-120303.0,7.65456,6.96496,6.27536,0.0000,4.90194


In [27]:
df2[ (df2.close >= df2.boll_ub) & (df2.close * 0.95  <= df2.boll_ub) & (df2.macdh > 0)& (df2.macd > 0)& (df2.macds > 0)]
# .loc[:,['rsi_6','rsi_12','rsi_24','rsi']]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
34,2023-08-15,000046,*ST泛海,1.76,0.139599,0.115891,0.023708,94.5158,89.1257,105.2960,...,0.241379,0.213659,2797730.0,4095280.0,2919410.0,1.70718,1.55338,1.39958,100.0000,90.9569
160,2023-08-15,000590,启迪药业,9.11,0.079590,0.047374,0.032216,69.3887,59.4775,89.2112,...,0.080422,0.039849,1820180.0,1272470.0,382889.0,9.73914,8.86174,7.98434,97.1736,81.2707
195,2023-08-15,000637,ST实华,3.51,0.025685,0.006779,0.018906,78.1386,72.7914,88.8329,...,0.161663,0.158304,65734.0,83868.2,58798.5,3.79287,3.45117,3.10947,87.0306,71.4174
231,2023-08-15,000701,厦门信达,7.14,0.097195,0.036738,0.060457,60.2088,52.9133,74.7998,...,0.201228,0.149342,21737300.0,14620900.0,5217090.0,7.40925,6.74175,6.07425,100.0000,95.1844
234,2023-08-15,000705,浙江震元,9.75,0.293813,0.136685,0.157128,75.3265,70.8999,84.1796,...,0.272116,0.184493,40389000.0,30846700.0,10010700.0,9.53934,8.67994,7.82054,100.0000,100.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26576,2023-08-22,603607,京华激光,15.59,0.225210,0.139567,0.085643,77.7822,67.8158,97.7150,...,0.191207,0.091777,1056620.0,1463820.0,825961.0,16.52010,15.03180,13.54350,100.0000,73.4584
26663,2023-08-22,603767,中马传动,15.27,0.406781,0.233536,0.173245,76.9720,61.6060,107.7040,...,0.076481,0.040364,68276900.0,47535000.0,17846600.0,14.62650,13.30880,11.99110,100.0000,92.1437
26689,2023-08-22,603817,海峡环保,6.59,0.053748,0.042504,0.011243,69.3123,57.9383,92.0602,...,0.077844,-0.017967,0.0,579551.0,375796.0,7.10955,6.46905,5.82855,100.0000,82.1143
26781,2023-08-22,603985,恒润股份,26.20,0.840355,0.683833,0.156521,69.3727,60.6695,86.7793,...,-0.004048,0.015058,22770200.0,16703800.0,7192130.0,26.97630,24.54600,22.11570,100.0000,82.3451


# inStock 序列化数据测试

## 定义参数

In [37]:
stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\2020'+get_today(sep='')[-4:-2]+'\\2020'+get_today(sep='')[-4:]
stock_hist_cache_path

'Y:\\dockerf\\stock\\instockcache\\hist\\202008\\20200822'

In [38]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
get_today()
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\2020'+get_today(sep='')[-4:]
code = '601336'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")


'2023-08-22'

Y:\dockerf\stock\instockcache\hist\202008\20200822\601336qfq.gzip.pickle


In [39]:
df.shape
df[-2:]
# df.columns
lastday=df.date.values[-1]
lastday
data = df.copy()
# macd
data.loc[:, 'diff'], data.loc[:, 'dea'], data.loc[:, 'macd'] = tl.MACD(
    data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['diff'].values[np.isnan(data['diff'].values)] = 0.0
data['dea'].values[np.isnan(data['dea'].values)] = 0.0
data['macd'].values[np.isnan(data['macd'].values)] = 0.0

data.loc[:, 'boll_ub'], data.loc[:, 'boll'], data.loc[:, 'boll_lb'] = tl.BBANDS \
    (data['close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
data['boll_ub'].values[np.isnan(data['boll_ub'].values)] = 0.0
data['boll'].values[np.isnan(data['boll'].values)] = 0.0
data['boll_lb'].values[np.isnan(data['boll_lb'].values)] = 0.0
data.iloc[-60:]['macd'].max()
# DIFF:=EMA(CLOSE,12) - EMA(CLOSE,26);  256 
# DEA := EMA(DIFF,9); 1.52
# MACD := 2*(DIFF-DEA); 2.08
print(not False)

(728, 11)

,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
726,2023-08-21,43.00,41.23,43.00,41.16,216448,899485217.0,4.30,-3.74,-1.60,1.04
727,2023-08-22,41.32,42.18,42.37,41.32,115327,483342073.0,2.55,2.30,0.95,0.55


'2023-08-22'

0.6059992411342487

True


## stock_period_to_typ Resample 'W-FRI'

In [40]:
def get_tdx_stock_period_to_type(stock_data, period_day='W-FRI', periods=5, ncol=None, ratiodays=True):
    """_周期转换周K,月K_

    Returns:
        _type_: _description_
    """
    #快速日期处理
    #https://www.likecs.com/show-204682607.html

    period_type = period_day
    if 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
    stock_data['date'] = stock_data.index
    lastday = stock_data.date.values[-1]
    lastday2 = stock_data.date.values[-2]
    indextype = True if stock_data.index.dtype == 'datetime64[ns]' else False
    # if cct.get_work_day_status() and 915 < cct.get_now_time_int() < 1500:
    #     stock_data = stock_data[stock_data.index < cct.get_today()]

    if stock_data.index.name == 'date':
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    elif 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
        stock_data.sort_index(ascending=True, inplace=True)
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    # else:
    #     log.error("index.name not date,pls check:%s" % (stock_data[:1]))

    period_stock_data = stock_data.resample(period_type).last()
    # period_stock_data['percent']=stock_data['percent'].resample(period_type,how=lambda x:(x+1.0).prod()-1.0)
    # print stock_data.index[0],stock_data.index[-1]
    # period_stock_data.index =
    # pd.DatetimeIndex(start=stock_data.index.values[0],end=stock_data.index.values[-1],freq='BM')

    period_stock_data['open'] = stock_data[
        'open'].resample(period_type).last()
    period_stock_data['high'] = stock_data[
        'high'].resample(period_type).max()
    period_stock_data['low'] = stock_data[
        'low'].resample(period_type).min()
    
    lastWeek1 = str(period_stock_data['open'].index.values[-1])[:10]
    lastweek2 = str(period_stock_data['open'].index.values[-2])[:10]
    if ratiodays:
        if period_day == 'W-FRI':
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])
            ratio_d =(5-(duratio%5))/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day.find('W') >= 0:
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastday, '%Y-%m-%d').date() - datetime.datetime.strptime(lastweek2, '%Y-%m-%d').date())[0])
            ratio_d =(duratio)/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day == 'BM':
            # daynow = '2023-04-26'
            # lastday = '2023-04-23'
            print(lastWeek1,lastweek2,lastday,lastday2)
            # print((str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2]))
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2])
            ratio_d =(30-(duratio%30))/30
            print("ratio_d:%s %s dura:%s"%(ratio_d,lastday,duratio))
        elif period_day.find('M') >= 0:
            ratio_d = 1
            
    else:
        ratio_d = 1
        print(ratio_d)
        
    if ncol is not None:
        for co in ncol:
            period_stock_data[co] = stock_data[co].resample(period_type).sum()
            if ratiodays:
                period_stock_data[co] = period_stock_data[co].apply(lambda x: round(x / ratio_d, 1))
                
    # else:
    period_stock_data['amount'] = stock_data[
        'amount'].resample(period_type).sum()
    period_stock_data['volume'] = stock_data[
        'volume'].resample(period_type).sum()
    if ratiodays and ratio_d != 1:
        period_stock_data['amount'] = period_stock_data['amount'].apply(lambda x: round(x / ratio_d, 1))
        period_stock_data['volume'] = period_stock_data['volume'].apply(lambda x: round(x / ratio_d, 1))
                
    # period_stock_data['turnover']=period_stock_data['vol']/(period_stock_data['traded_market_value'])/period_stock_data['close']
    period_stock_data.index = stock_data['date'].resample(period_type).last().index
    # print period_stock_data.index[:1]
    if 'code' in period_stock_data.columns:
        period_stock_data = period_stock_data[period_stock_data['code'].notnull()]
    period_stock_data = period_stock_data.dropna()
    # period_stock_data.reset_index(inplace=True)
    # period_stock_data.set_index('date',inplace=True)
    # print period_stock_data.columns,period_stock_data.index.name
    # and period_stock_data.index.dtype != 'datetime64[ns]')
    if not indextype and period_stock_data.index.name == 'date':
        # stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
        period_stock_data.index = [str(x)[:10] for x in period_stock_data.index]
        period_stock_data.index.name = 'date'
    else:
        if 'date' in period_stock_data.columns:
            period_stock_data = period_stock_data.drop(['date'], axis=1)
    # print period_stock_data
    return period_stock_data


### Resample Week W-FRI

In [41]:
# dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
# dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
# data[-5:]

2023-08-25 2023-08-18 2023-08-22 2023-08-21
ratio_d:0.4 2023-08-22


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-08-11,44.47,42.85,45.45,42.68,1936922.5,8.643637e+09,40.9,-15.8,-1.81,9.3,1.658863,1.727008,-0.068144,47.755362,42.0325,36.309638,2023-08-11
2023-08-18,42.34,42.83,44.66,41.90,1677222.5,7.212552e+09,41.2,-0.1,0.48,8.1,0.962706,1.346325,-0.383619,46.975172,43.1395,39.303828,2023-08-18
2023-08-25,41.32,42.18,43.00,41.16,829437.5,3.457068e+09,17.1,-3.6,0.95,4.0,0.642734,1.109485,-0.466751,46.174024,43.3890,40.603976,2023-08-22


2023-08-27 2023-08-20 2023-08-22 2023-08-21
ratio_d:0.4 2023-08-22


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-08-13,44.47,42.85,45.45,42.68,1936922.5,8.643637e+09,40.9,-15.8,-1.81,9.3,1.658863,1.727008,-0.068144,47.755362,42.0325,36.309638,2023-08-11
2023-08-20,42.34,42.83,44.66,41.90,1677222.5,7.212552e+09,41.2,-0.1,0.48,8.1,0.962706,1.346325,-0.383619,46.975172,43.1395,39.303828,2023-08-18
2023-08-27,41.32,42.18,43.00,41.16,829437.5,3.457068e+09,17.1,-3.6,0.95,4.0,0.642734,1.109485,-0.466751,46.174024,43.3890,40.603976,2023-08-22


### Resample Month BM
https://note.nkmk.me/python-pandas-time-series-freq/

    BM: 月末最終営業日ごと
    MS: 毎月の開始日ごと
    BMS: 毎月の最初の営業日ごと


In [42]:
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
dfw[-2:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]

1


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-07-31,44.47,44.34,45.60,35.71,3485081,1.437417e+10,74.47,22.96,-0.16,16.73,1.476466,0.908265,0.568201,43.363012,39.431,35.498988,2023-07-31
2023-08-31,41.32,42.18,48.03,41.16,2775397,1.236552e+10,58.72,-4.64,0.95,13.30,0.642734,1.109485,-0.466751,46.174024,43.389,40.603976,2023-08-22


2023-08-31 2023-07-31 2023-08-22 2023-08-21
ratio_d:0.7 2023-08-22 dura:9


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-06-30,35.22,35.69,38.76,33.90,3457402.9,1.294795e+10,87.5,0.2,0.39,16.6,-0.285216,-0.096784,-0.188432,38.074762,36.1565,34.238238,2023-06-30
2023-07-31,44.47,44.34,45.60,35.71,4978687.1,2.053453e+10,106.4,32.8,-0.16,23.9,1.476466,0.908265,0.568201,43.363012,39.4310,35.498988,2023-07-31
2023-08-31,41.32,42.18,48.03,41.16,3964852.9,1.766504e+10,83.9,-6.6,0.95,19.0,0.642734,1.109485,-0.466751,46.174024,43.3890,40.603976,2023-08-22


In [43]:
daynow=dfw.index.values[-1]
str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0]
int(str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])


'9'

9

# inStock RSI

In [44]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\20200530'
code = '000002'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")

Y:\dockerf\stock\instockcache\hist\202008\20200822\000002qfq.gzip.pickle


In [45]:
df.shape
df[-5:]
df[:1]
# df.columns
lastday=df.date.values[-1]
lastday

(728, 11)

,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
723,2023-08-16,14.25,14.54,14.78,14.21,789837,1.147494e+09,3.99,1.75,0.25,0.81
724,2023-08-17,14.41,14.53,14.64,14.30,705984,1.019721e+09,2.34,-0.07,-0.01,0.73
725,2023-08-18,14.51,14.31,14.67,14.31,550032,7.966413e+08,2.48,-1.51,-0.22,0.57
726,2023-08-21,14.27,13.98,14.29,13.98,594691,8.399745e+08,2.17,-2.31,-0.33,0.61
727,2023-08-22,13.99,14.02,14.11,13.88,458812,6.420014e+08,1.65,0.29,0.04,0.47


,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
0,2020-08-24,25.35,24.9,25.52,24.87,566269,1.549031e+09,2.59,-0.92,-0.23,0.58


'2023-08-22'

In [46]:
data=df.copy()

In [47]:
def check_rsi_status(df,threshold=60,period_day=False):
    # macd
    data = df.copy()
    # cci 计算方法和结果和stockstats不同，stockstats典型价采用均价(总额/成交量)计算
    data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
    data['cci'].values[np.isnan(data['cci'].values)] = 0.0
    # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
    # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
    rsilast5 = data.iloc[-6:-1]['cci']
    rsilast5max = rsilast5.max()
    rsilast = data.iloc[-1]['cci']

    if rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max:
        return True
    else:
        return False

In [48]:
# data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
# data['cci'].values[np.isnan(data['cci'].values)] = 0.0
# # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
# # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
# rsilast5 = data.iloc[-6:-1]['cci']
# rsilast5max = rsilast5.max()
# rsilast = data.iloc[-1]['cci']
# data['cci'][-6:]
# rsilast5.iloc[-1]
# rsilast5,rsilast
# rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max
check_rsi_status(df)

False